# Calculate a risk assessment of the Logistic Regression model

In [1]:
!pip install --upgrade pip
!pip install pandas

In [1]:
import numpy as np
import pandas as pd
import pickle

# add folder with custom functions to path
import sys
sys.path.append('../onc_functions')
from risk import get_risk_categories

print('python-' + sys.version)
import datetime
dte = datetime.datetime.now()
dte = dte.strftime("%Y%m%d")

python-3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


# Risk categories

In [2]:
# here' we import the dataset we created for the fairness assessment in notebook 6
with open('complete_fair1.pickle','rb') as f:  
    details = pickle.load(f)
details.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset
0,2969676.0,0.0,72.0,2.0,1.0,2.0,2.0,8
1,2969869.0,1.0,73.0,2.0,1.0,2.0,2.0,9
2,2970021.0,0.0,55.0,2.0,1.0,1.0,1.0,8
3,2970067.0,0.0,72.0,1.0,1.0,1.0,2.0,9
4,2970686.0,0.0,49.0,2.0,1.0,2.0,2.0,8


In [3]:
with open('./results/2021_final_LR_model_test_pred_proba_pooled.pickle','rb') as f:  
    y_pred = pickle.load(f)
    
y_pred.head()    

,imp1,imp2,imp3,imp4,imp5,score,std_,usrds_id,died_in_90,subset
0,0.925393,0.914949,0.901626,0.929962,0.921700,0.918726,0.009861,31089.0,1.0,9
1,0.778438,0.800176,0.776335,0.791798,0.777974,0.784944,0.009424,34521.0,0.0,8
2,0.449145,0.417679,0.474578,0.594807,0.488599,0.484961,0.059995,46751.0,0.0,8
3,0.636240,0.627138,0.649013,0.598810,0.635832,0.629407,0.016815,50506.0,0.0,9
4,0.168010,0.186585,0.171646,0.276738,0.167967,0.194189,0.041841,54985.0,0.0,8


In [4]:
data = details.merge(y_pred, on=['usrds_id','died_in_90','subset'])
data.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset,imp1,imp2,imp3,imp4,imp5,score,std_
0,2969676.0,0.0,72.0,2.0,1.0,2.0,2.0,8,0.524443,0.460429,0.432501,0.520211,0.453502,0.478217,0.037195
1,2969869.0,1.0,73.0,2.0,1.0,2.0,2.0,9,0.492358,0.492342,0.492408,0.492367,0.492423,0.492380,0.000031
2,2970021.0,0.0,55.0,2.0,1.0,1.0,1.0,8,0.451538,0.451482,0.451440,0.451575,0.451419,0.451491,0.000059
3,2970067.0,0.0,72.0,1.0,1.0,1.0,2.0,9,0.058317,0.058300,0.058325,0.058311,0.058305,0.058312,0.000009
4,2970686.0,0.0,49.0,2.0,1.0,2.0,2.0,8,0.216587,0.216604,0.216638,0.216629,0.216636,0.216619,0.000020


In [8]:
risk_cat = get_risk_categories(data,
                               y_proba_col_name='score',
                               y_true_col_name='died_in_90')
risk_cat

,Risk Category,Count,Count Died in 90,Count Survived,Percent Died in 90
0,0-0.09,43117,145.0,42972.0,0.003363
1,0.1-0.19,56041,633.0,55408.0,0.011295
2,0.2-0.29,51733,1103.0,50630.0,0.021321
3,0.3-0.39,44494,1770.0,42724.0,0.039781
4,0.4-0.49,38156,2412.0,35744.0,0.063214
5,0.5-0.59,33358,3208.0,30150.0,0.096169
6,0.6-0.69,29338,4074.0,25264.0,0.138864
7,0.7-0.79,24342,4765.0,19577.0,0.195752
8,0.8-0.89,17561,4878.0,12683.0,0.277775
9,0.9-0.99,7141,2740.0,4401.0,0.383700


In [6]:
risk_cat.to_csv('./results/' + str(dte) + '_lr_risk_cat.csv')